In [2]:
import numpy as np
import pandas as pd
import scanpy as sc
from cmapPy.pandasGEXpress.parse import parse
import h5py

In [3]:
df_gene = pd.read_csv('/path/to/geneinfo_processed.csv')

In [4]:
df_landmark_gene = df_gene[(df_gene['pr_is_bing'] == 1) & (df_gene['pr_is_lm']==1)]
df_best_infer_gene = df_gene[(df_gene['pr_is_bing'] == 1) & (df_gene['pr_is_lm']==0)]
landmark_ids = df_landmark_gene['pr_id'].tolist()
best_infer_ids = df_best_infer_gene['pr_id'].tolist()

In [5]:
weight_path = '/path/to/infer_weight.gctx'
infer_weight = parse(weight_path, cid=['OFFSET']+landmark_ids, rid=best_infer_ids)
infer_weight_df_tmp = infer_weight.data_df
infer_weight_df = infer_weight_df_tmp[['OFFSET'] + landmark_ids]
infer_weight_df = infer_weight_df.loc[best_infer_ids]

In [6]:
adata = sc.read_h5ad("../level3_cp_ctrl.pred.h5ad")

In [7]:
pred_inferred = np.dot(adata.layers['pred'], np.array(infer_weight_df.T[1:])) + np.array(
    infer_weight_df.T.loc['OFFSET'])  # (18539, 9196)

In [14]:
unpert_inferred = np.dot(adata.layers['unpert_expr'], np.array(infer_weight_df.T[1:])) + np.array(
    infer_weight_df.T.loc['OFFSET'])  # (18539, 9196)

In [9]:
pert_inferred = np.dot(adata.X, np.array(infer_weight_df.T[1:])) + np.array(
    infer_weight_df.T.loc['OFFSET'])  # (18539, 9196)

In [11]:
pred_inferred.shape

(77895, 9196)

In [12]:
pred_all = np.concatenate((adata.layers['pred'], pred_inferred), axis=1)

In [15]:
unpert_all = np.concatenate((adata.layers['unpert_expr'], unpert_inferred), axis=1)

In [16]:
pert_all = np.concatenate((adata.X, pert_inferred), axis=1)

In [18]:
adata.var_names

Index(['GNPDA1', 'CDH3', 'HDAC6', 'PARP2', 'MAMLD1', 'DNAJB6', 'SMC4', 'ABCC5',
       'ABCB6', 'DNM1L',
       ...
       'NCAPD2', 'PAN2', 'LPGAT1', 'KIF14', 'CDC25A', 'CDC25B', 'OXSR1', 'MVP',
       'CDC42', 'DMTF1'],
      dtype='object', length=978)

In [19]:
df_gene

,pr_id,pr_gene_id,pr_gene_symbol,pr_gene_title,pr_is_lm,pr_is_bing,gene_id
0,202382_s_at,10007,GNPDA1,glucosamine-6-phosphate deaminase 1,1,1,10007
1,203256_at,1001,CDH3,cadherin 3,1,1,1001
2,206846_s_at,10013,HDAC6,histone deacetylase 6,1,1,10013
3,214086_s_at,10038,PARP2,poly(ADP-ribose) polymerase 2,1,1,10038
4,205088_at,10046,MAMLD1,mastermind like domain containing 1,1,1,10046
...,...,...,...,...,...,...,...
10169,201131_s_at,999,CDH1,cadherin 1,0,1,999
10170,220740_s_at,9990,SLC12A6,solute carrier family 12 (potassium/chloride t...,0,1,9990
10171,214697_s_at,9991,PTBP3,polypyrimidine tract binding protein 3,0,1,9991
10172,214198_s_at,9993,DGCR2,DiGeorge syndrome critical region gene 2,0,1,9993


In [22]:
import anndata as ad

In [23]:
X = pert_all  
layer_unpert = unpert_all
layer_pred = pred_all

adata1 = ad.AnnData(X=X)

adata1.layers["unpert_expr"] = layer_unpert
adata1.layers["pred"] = layer_pred

adata1.var_names = df_gene["pr_gene_symbol"].astype(str).values

In [24]:
adata1.obs = adata.obs

In [28]:
adata1.write_h5ad("level3_cp_ctrl.pred.infer_genes.h5ad")